# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Разбор : влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Признаки


- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита.





Предобработка данных:
- Аномалии;
- Корректировка типов данных;
- Дубликаты;
- Пропущенные значения.

Проверка гипотез:
- зависимость между наличием детей и возвратом кредита в срок;
- зависимость между семейным положением и возвратом кредита в срок;
- зависимость между уровнем дохода и возвратом кредита в срок;
- Цели кредита и возврат кредита в срок;


Подготовка и предварительный осмотр


In [1]:
# %pip install pymorphy2

In [2]:
# %pip install pandas-profiling

In [3]:
# %pip uninstall markupsafe
# %pip install markupsafe==2.0.1

In [4]:
# Ваш код здесь
import pandas as pd
import numpy as np
from scipy import stats as st
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import math 
import statistics
# pd.set_option('display.max_columns', None)
# pd.options.display.max_colwidth = 150
from pandas_profiling import ProfileReport

# profile = ProfileReport(df, title='Профиль_Пандас_Репорт')
import pandas_profiling as pp


import nltk # библиотека nltk
from nltk.tokenize import word_tokenize, sent_tokenize # готовые токенизаторы библиотеки nltk
from pymorphy2 import MorphAnalyzer

# импортируем стоп-слова из библиотеки nltk
from nltk.corpus import stopwords


In [5]:
df = pd.read_csv('data.csv',sep=',')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [6]:
df.duplicated().sum()

54

In [7]:
df = df.drop_duplicates()
df.duplicated().sum()

0

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21471 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21471 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21471 non-null  int64  
 3   education         21471 non-null  object 
 4   education_id      21471 non-null  int64  
 5   family_status     21471 non-null  object 
 6   family_status_id  21471 non-null  int64  
 7   gender            21471 non-null  object 
 8   income_type       21471 non-null  object 
 9   debt              21471 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21471 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Переведём текст на русский язык приятный для глаз.

In [9]:
dict_to_rus = {
    'children': 'количество детей в семье',
    'days_employed':'общий трудовой стаж в днях',
    'dob_years': 'возраст клиента в годах',
    'education': 'уровень образования клиента',
    'education_id': 'идентификатор уровня образования',
    'family_status': 'семейное положение',
    'family_status_id': 'идентификатор семейного положения',
    'gender':'пол',
    'income_type':'тип занятости',
    'debt': 'имел ли задолженность по возврату кредитов',
    'total_income':'ежемесячный доход',
    'purpose':'цель получения кредита'
}

df.rename(columns = (dict_to_rus), inplace=True)

Cделаем функцию вывода уникальных значений и пропусков, чтобы обращаться к ней, если нам понадобиться

In [10]:
def out_df_uniq_propusk(df):
    for i in df:
        print('*'*125)
        print('\n')
        print('Столбец : ', i)
        print('уникальные значения : ', df[i].unique())
        print('\n')
        print('пропуски : ', df[i].isnull().sum())
        print('\n')
        print('Описательная статистика : ')
        print(df[i].describe())
        print('\n')
        print('Количество значений : ')
        print(df[i].value_counts())
        print('\n')

out_df_uniq_propusk(df)

*****************************************************************************************************************************


Столбец :  количество детей в семье
уникальные значения :  [ 1  0  3  2 -1  4 20  5]


пропуски :  0


Описательная статистика : 
count    21471.000000
mean         0.539565
std          1.382978
min         -1.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: количество детей в семье, dtype: float64


Количество значений : 
 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: количество детей в семье, dtype: int64


*****************************************************************************************************************************


Столбец :  общий трудовой стаж в днях
уникальные значения :  [-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]


пропуски :  2120


Описательная статистика : 
count     

Из уникальных значений и их количеств, а также пропусков, поставим задачи для подготовки данных. 

В признаке:

1)"количество детей в семье", присутствует значение : -1 количеством 47 и 20 количеством 76.

2)"уровень образования клиента" категориальный признак с разными регистрами.

3)"пол", присуствует третье значение 'XNA'. Это одна запись, будет удалена.

4)"общий трудовой стаж в днях", имеются отрицательные значения. 

5)"цель получения кредита" требуется провести категоризацию признака

6)"возраст клиента в годах" присутствует значение 0 количеством 101

7)"общий трудовой стаж в днях" и "ежемесячный доход" имеют 2120 пропусков.

8)"общий трудовой стаж в днях" аномальные значения, не соответствуют действительности.

9)"идентификатор уровня образования" и "идентификатор семейного положения" соответствуют категориальным признакам "уровень образования клиента" и "семейное положение"



Начнём с первого. "количество детей в семье", присутствует значение : -1 количеством 47 и 20 количеством 76.

Для подробного исследования, воспользуемся полезной библиотекой pandas-profilling

In [11]:
# profile= pp.ProfileReport(df)
profile = ProfileReport(df, title="Pandas Profiling Report")

# profile.to_file('pandas_profile_test.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
profile.to_file('инфо.html')

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

В файле отчёта 'инфо.html' есть дополнительная информация: высокая корреляция между семейным положением и целью получения кредита, возраст клиента коррелирует с трудовым стажем, тип занятости коррелирует со стажем. 

47 значений -1, не так много. Преобразуем те записи, в колонке у которых количество детей -1, в 1. А те, что 20 добавим их к числу тех, у которых 2 детей, судя по характеристикам в файле, отличий критичных не наблюдается.

In [13]:
df[df['количество детей в семье'] == 20].describe()

,количество детей в семье,общий трудовой стаж в днях,возраст клиента в годах,идентификатор уровня образования,идентификатор семейного положения,имел ли задолженность по возврату кредитов,ежемесячный доход
count,76.0,67.000000,76.000000,76.000000,76.000000,76.000000,67.000000
mean,20.0,35779.535997,41.815789,0.842105,0.815789,0.105263,168720.526180
std,0.0,112008.279425,12.065058,0.433671,1.363432,0.308931,80851.738318
min,20.0,-11937.171389,0.000000,0.000000,0.000000,0.000000,53971.576721
25%,20.0,-2629.663890,33.750000,1.000000,0.000000,0.000000,114668.631192
50%,20.0,-957.251278,41.500000,1.000000,0.000000,0.000000,144544.530170
75%,20.0,-478.364082,50.000000,1.000000,1.000000,0.000000,218033.697152
max,20.0,385267.263676,69.000000,2.000000,4.000000,1.000000,441721.334145


In [14]:
df[df['количество детей в семье'] == 2].describe()

,количество детей в семье,общий трудовой стаж в днях,возраст клиента в годах,идентификатор уровня образования,идентификатор семейного положения,имел ли задолженность по возврату кредитов,ежемесячный доход
count,2052.0,1851.000000,2052.000000,2052.000000,2052.000000,2052.000000,1.851000e+03
mean,2.0,1274.234500,35.769493,0.788986,0.451267,0.094542,1.718522e+05
std,0.0,35475.341365,7.226231,0.556698,0.971090,0.292652,1.072317e+05
min,2.0,-13039.072024,0.000000,0.000000,0.000000,0.000000,2.809288e+04
25%,2.0,-2896.291250,31.000000,0.000000,0.000000,0.000000,1.023172e+05
50%,2.0,-1634.342227,35.000000,1.000000,0.000000,0.000000,1.446486e+05
75%,2.0,-775.697378,40.000000,1.000000,1.000000,0.000000,2.092035e+05
max,2.0,397548.767244,64.000000,3.000000,4.000000,1.000000,1.103455e+06


In [15]:
df[df['количество детей в семье'] == -1] = 1
df[df['количество детей в семье'] == 20] = 2

In [16]:
df[df['количество детей в семье'] == 20]

,количество детей в семье,общий трудовой стаж в днях,возраст клиента в годах,уровень образования клиента,идентификатор уровня образования,семейное положение,идентификатор семейного положения,пол,тип занятости,имел ли задолженность по возврату кредитов,ежемесячный доход,цель получения кредита


Проверим на дубликаты и заметим, что появились дубликаты, с значениями 1 и 2, по всем признакам. Удалим их.

In [17]:
dupl = df[df.duplicated()]
df.drop(index = dupl.index, inplace=True)

In [18]:
df.duplicated().sum()

0

In [19]:
df.drop(index=df[df['уровень образования клиента'] == 1].index, inplace=True)
df.drop(index=df[df['уровень образования клиента'] == 2].index, inplace=True)

3) Унифицируем Уровень образования

In [20]:
df['уровень образования клиента'] = df['уровень образования клиента'].str.lower()
df['уровень образования клиента'].value_counts()

среднее                15091
высшее                  5228
неоконченное высшее      741
начальное                282
ученая степень             6
Name: уровень образования клиента, dtype: int64

3) Удалим надоедливую одну запись со значением поля XNA.

In [21]:
df.drop(index=df[df['пол'] == 'XNA'].index, inplace=True)
df[df['пол'] == 'XNA']

,количество детей в семье,общий трудовой стаж в днях,возраст клиента в годах,уровень образования клиента,идентификатор уровня образования,семейное положение,идентификатор семейного положения,пол,тип занятости,имел ли задолженность по возврату кредитов,ежемесячный доход,цель получения кредита


Видны записи по 2 и одной у по типам занятости : студент, безработынй и в декрете. удалим их.

In [22]:
type_job = ['безработный','студент','в декрете','предприниматель']

for i in type_job:
    df.drop(index= df[df['тип занятости'] == i].index, inplace=True)

Теперь посмотрим характеристики

In [23]:
out_df_uniq_propusk(df)
print(df.info())

*****************************************************************************************************************************


Столбец :  количество детей в семье
уникальные значения :  [1 0 3 2 4 5]


пропуски :  0


Описательная статистика : 
count    21341.000000
mean         0.473689
std          0.752206
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          5.000000
Name: количество детей в семье, dtype: float64


Количество значений : 
0    14102
1     4808
2     2051
3      330
4       41
5        9
Name: количество детей в семье, dtype: int64


*****************************************************************************************************************************


Столбец :  общий трудовой стаж в днях
уникальные значения :  [-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]


пропуски :  2107


Описательная статистика : 
count     19234.000000
mean      63141.773911
std      14091

4) Преобразуем значения "общий трудовой стаж" в абсолютные значения. Избавимся от минуса.

In [24]:
df['общий трудовой стаж в днях'] = df['общий трудовой стаж в днях'].round().abs()

5) Проведём категоризацию признака 'цель получения кредита'

Токенизация и леммитизация

In [25]:
nltk.download('punkt') #пунктуация
nltk.download('stopwords') #стоп-слова

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bair\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bair\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
analyzer = MorphAnalyzer()

# посмотрим на стоп-слова для русского языка
stop_words = stopwords.words('russian')

In [27]:
df['цель получения кредита'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [28]:
def tok_clear_stop_words_and_lemm(words):
    clear_words_tok_no_stop_lemm=[]
    words = word_tokenize(words)
    for word in words:
        if word not in stop_words:
            clear_words_tok_no_stop_lemm.append(analyzer.parse(word)[0].normal_form)
    return clear_words_tok_no_stop_lemm

In [29]:
df['лемматизация'] = df['цель получения кредита'].apply(tok_clear_stop_words_and_lemm)
df['лемматизация'].value_counts()

[автомобиль]                                  966
[свадьба]                                     792
[проведение, свадьба]                         767
[сыграть, свадьба]                            764
[операция, недвижимость]                      672
[покупка, коммерческий, недвижимость]         659
[покупка, жильё, сдача]                       649
[операция, жильё]                             647
[операция, коммерческий, недвижимость]        645
[жильё]                                       641
[покупка, жильё]                              640
[покупка, жильё, семья]                       637
[недвижимость]                                631
[строительство, собственный, недвижимость]    627
[операция, свой, недвижимость]                623
[строительство, жилой, недвижимость]          621
[покупка, свой, жильё]                        619
[строительство, недвижимость]                 619
[покупка, недвижимость]                       615
[ремонт, жильё]                               603


In [30]:
def purpose_to_cat(lemmas):
    cat_cred = ''
    if ('жилье' in lemmas) or ('жильё' in lemmas) or ('недвижимость' in lemmas):
        cat_cred = 'жилищное кредитование'
    if 'коммерческий' in lemmas:
        cat_cred = 'развитие бизнеса'
    if ('свадьба' in lemmas) or ('строительство' in lemmas) or ('ремонт' in lemmas):
        cat_cred = 'потребительский кредит'
    if ('образование' in lemmas):
        cat_cred = 'образовательный кредит'
    if ('автомобиль' in lemmas):
        cat_cred = 'автокредитование'
    return cat_cred

df['категория кредита'] = df['лемматизация'].apply(purpose_to_cat)

In [31]:
df['категория кредита'] = df['лемматизация'].apply(purpose_to_cat)

In [32]:
df['категория кредита'].value_counts()

жилищное кредитование     6975
потребительский кредит    4793
автокредитование          4280
образовательный кредит    3989
развитие бизнеса          1304
Name: категория кредита, dtype: int64

In [33]:
df.drop(columns= 'лемматизация', inplace=True)

In [34]:
df.drop(columns='цель получения кредита',inplace=True)

Создадим дополнительный файлик информации нашего датафрэйма для просмотра статистики.

In [35]:
profile_posle_1 = ProfileReport(df, title="Pandas Profiling Report")


In [36]:
profile_posle_1.to_file('инфо_2.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

найдём дубликаты

In [37]:
df.duplicated().sum()

258

In [38]:
df = df.drop_duplicates()

In [39]:
df.duplicated().sum()

0

6) Найдём записи у которых возраст, равен 0

In [40]:
df[df['возраст клиента в годах'] == 0]

,количество детей в семье,общий трудовой стаж в днях,возраст клиента в годах,уровень образования клиента,идентификатор уровня образования,семейное положение,идентификатор семейного положения,пол,тип занятости,имел ли задолженность по возврату кредитов,ежемесячный доход,категория кредита
99,0,346542.0,0,среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автокредитование
149,0,2664.0,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,жилищное кредитование
270,3,1873.0,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,потребительский кредит
578,0,397857.0,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,потребительский кредит
1040,0,1158.0,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,автокредитование
...,...,...,...,...,...,...,...,...,...,...,...,...
19371,1,2356.0,0,высшее,0,женат / замужем,0,F,компаньон,0,77936.218285,автокредитование
20462,0,338735.0,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,жилищное кредитование
20577,0,331741.0,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,жилищное кредитование
21179,2,109.0,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,потребительский кредит


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21083 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   количество детей в семье                    21083 non-null  int64  
 1   общий трудовой стаж в днях                  19234 non-null  float64
 2   возраст клиента в годах                     21083 non-null  int64  
 3   уровень образования клиента                 21083 non-null  object 
 4   идентификатор уровня образования            21083 non-null  int64  
 5   семейное положение                          21083 non-null  object 
 6   идентификатор семейного положения           21083 non-null  int64  
 7   пол                                         21083 non-null  object 
 8   тип занятости                               21083 non-null  object 
 9   имел ли задолженность по возврату кредитов  21083 non-null  int64  
 10  ежемесячны

In [42]:
s = df[df['возраст клиента в годах'] == 0]

In [43]:
s['возраст клиента в годах'] = np.nan

C:\Users\Bair\AppData\Local\Temp\ipykernel_6624\3229382100.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s['возраст клиента в годах'] = np.nan


In [44]:
s

,количество детей в семье,общий трудовой стаж в днях,возраст клиента в годах,уровень образования клиента,идентификатор уровня образования,семейное положение,идентификатор семейного положения,пол,тип занятости,имел ли задолженность по возврату кредитов,ежемесячный доход,категория кредита
99,0,346542.0,NaN,среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автокредитование
149,0,2664.0,NaN,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,жилищное кредитование
270,3,1873.0,NaN,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,потребительский кредит
578,0,397857.0,NaN,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,потребительский кредит
1040,0,1158.0,NaN,высшее,0,в разводе,3,F,компаньон,0,303994.134987,автокредитование
...,...,...,...,...,...,...,...,...,...,...,...,...
19371,1,2356.0,NaN,высшее,0,женат / замужем,0,F,компаньон,0,77936.218285,автокредитование
20462,0,338735.0,NaN,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,жилищное кредитование
20577,0,331741.0,NaN,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,жилищное кредитование
21179,2,109.0,NaN,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,потребительский кредит


In [45]:
df1 = pd.concat([s, df], ignore_index=True)
df1

,количество детей в семье,общий трудовой стаж в днях,возраст клиента в годах,уровень образования клиента,идентификатор уровня образования,семейное положение,идентификатор семейного положения,пол,тип занятости,имел ли задолженность по возврату кредитов,ежемесячный доход,категория кредита
0,0,346542.0,NaN,среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автокредитование
1,0,2664.0,NaN,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,жилищное кредитование
2,3,1873.0,NaN,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,потребительский кредит
3,0,397857.0,NaN,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,потребительский кредит
4,0,1158.0,NaN,высшее,0,в разводе,3,F,компаньон,0,303994.134987,автокредитование
...,...,...,...,...,...,...,...,...,...,...,...,...
21177,1,4529.0,43.0,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,жилищное кредитование
21178,0,343937.0,67.0,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,автокредитование
21179,1,2113.0,38.0,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,жилищное кредитование
21180,3,3112.0,38.0,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,автокредитование


In [46]:
df1[df1['возраст клиента в годах'] == 0]

,количество детей в семье,общий трудовой стаж в днях,возраст клиента в годах,уровень образования клиента,идентификатор уровня образования,семейное положение,идентификатор семейного положения,пол,тип занятости,имел ли задолженность по возврату кредитов,ежемесячный доход,категория кредита
198,0,346542.0,0.0,среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автокредитование
248,0,2664.0,0.0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,жилищное кредитование
369,3,1873.0,0.0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,потребительский кредит
675,0,397857.0,0.0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,потребительский кредит
1129,0,1158.0,0.0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,автокредитование
...,...,...,...,...,...,...,...,...,...,...,...,...
19100,1,2356.0,0.0,высшее,0,женат / замужем,0,F,компаньон,0,77936.218285,автокредитование
20147,0,338735.0,0.0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,жилищное кредитование
20261,0,331741.0,0.0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,жилищное кредитование
20848,2,109.0,0.0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,потребительский кредит


In [47]:
df1.drop(index=df1[df1['возраст клиента в годах'] == 0].index, inplace=True)

In [48]:
df1[df1['возраст клиента в годах'].isna()]

,количество детей в семье,общий трудовой стаж в днях,возраст клиента в годах,уровень образования клиента,идентификатор уровня образования,семейное положение,идентификатор семейного положения,пол,тип занятости,имел ли задолженность по возврату кредитов,ежемесячный доход,категория кредита
0,0,346542.0,NaN,среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автокредитование
1,0,2664.0,NaN,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,жилищное кредитование
2,3,1873.0,NaN,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,потребительский кредит
3,0,397857.0,NaN,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,потребительский кредит
4,0,1158.0,NaN,высшее,0,в разводе,3,F,компаньон,0,303994.134987,автокредитование
...,...,...,...,...,...,...,...,...,...,...,...,...
94,1,2356.0,NaN,высшее,0,женат / замужем,0,F,компаньон,0,77936.218285,автокредитование
95,0,338735.0,NaN,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,жилищное кредитование
96,0,331741.0,NaN,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,жилищное кредитование
97,2,109.0,NaN,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,потребительский кредит


In [49]:
df1.isna().sum()

количество детей в семье                         0
общий трудовой стаж в днях                    1849
возраст клиента в годах                         99
уровень образования клиента                      0
идентификатор уровня образования                 0
семейное положение                               0
идентификатор семейного положения                0
пол                                              0
тип занятости                                    0
имел ли задолженность по возврату кредитов       0
ежемесячный доход                             1849
категория кредита                                0
dtype: int64

Заполним пропущенные значение возраста.

In [50]:
df1

,количество детей в семье,общий трудовой стаж в днях,возраст клиента в годах,уровень образования клиента,идентификатор уровня образования,семейное положение,идентификатор семейного положения,пол,тип занятости,имел ли задолженность по возврату кредитов,ежемесячный доход,категория кредита
0,0,346542.0,NaN,среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автокредитование
1,0,2664.0,NaN,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,жилищное кредитование
2,3,1873.0,NaN,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,потребительский кредит
3,0,397857.0,NaN,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,потребительский кредит
4,0,1158.0,NaN,высшее,0,в разводе,3,F,компаньон,0,303994.134987,автокредитование
...,...,...,...,...,...,...,...,...,...,...,...,...
21177,1,4529.0,43.0,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,жилищное кредитование
21178,0,343937.0,67.0,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,автокредитование
21179,1,2113.0,38.0,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,жилищное кредитование
21180,3,3112.0,38.0,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,автокредитование


In [51]:
from sklearn.impute import KNNImputer
from math import nan

imp_nan = KNNImputer(missing_values=np.nan, n_neighbors=5)

In [52]:
ur = df1['уровень образования клиента']

In [53]:
ur

0        среднее
1        среднее
2        среднее
3        среднее
4         высшее
          ...   
21177    среднее
21178    среднее
21179    среднее
21180    среднее
21181    среднее
Name: уровень образования клиента, Length: 21083, dtype: object

In [54]:
df1['уровень образования клиента'].isnull().sum()

0

In [55]:
df1.drop(columns='уровень образования клиента', inplace=True)

In [56]:
semya = df1['семейное положение']

In [57]:
semya

0         женат / замужем
1               в разводе
2         женат / замужем
3         женат / замужем
4               в разводе
               ...       
21177    гражданский брак
21178     женат / замужем
21179    гражданский брак
21180     женат / замужем
21181     женат / замужем
Name: семейное положение, Length: 21083, dtype: object

In [58]:
df1.drop(columns='семейное положение', inplace=True)

In [59]:
df1['пол'].replace({'F': 0, 'M':1}, inplace=True) 

In [60]:
df1['тип занятости'].replace({'сотрудник': 0, 'компаньон':1, 'госслужащий': 2, 'пенсионер': 3}, inplace=True)

In [61]:
df1['категория кредита'].replace({'жилищное кредитование':0, 'потребительский кредит':1, 'автокредитование':2, 'образовательный кредит':3, 'развитие бизнеса':4}, inplace=True)

In [62]:
df1['категория кредита'].value_counts()

0    6856
1    4741
2    4244
3    3939
4    1303
Name: категория кредита, dtype: int64

In [63]:
df1.columns

Index(['количество детей в семье', 'общий трудовой стаж в днях',
       'возраст клиента в годах', 'идентификатор уровня образования',
       'идентификатор семейного положения', 'пол', 'тип занятости',
       'имел ли задолженность по возврату кредитов', 'ежемесячный доход',
       'категория кредита'],
      dtype='object')

In [64]:
df1

,количество детей в семье,общий трудовой стаж в днях,возраст клиента в годах,идентификатор уровня образования,идентификатор семейного положения,пол,тип занятости,имел ли задолженность по возврату кредитов,ежемесячный доход,категория кредита
0,0,346542.0,NaN,1,0,0,3,0,71291.522491,2
1,0,2664.0,NaN,1,3,0,0,0,70176.435951,0
2,3,1873.0,NaN,1,0,0,0,0,102166.458894,1
3,0,397857.0,NaN,1,0,0,3,0,97620.687042,1
4,0,1158.0,NaN,0,3,0,1,0,303994.134987,2
...,...,...,...,...,...,...,...,...,...,...
21177,1,4529.0,43.0,1,1,0,1,0,224791.862382,0
21178,0,343937.0,67.0,1,0,0,3,0,155999.806512,2
21179,1,2113.0,38.0,1,1,1,0,1,89672.561153,0
21180,3,3112.0,38.0,1,0,1,0,1,244093.050500,2


In [65]:
result = imp_nan.fit_transform(df1)

In [66]:
df2 = pd.DataFrame(data=result, columns= df1.columns)

In [67]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21083 entries, 0 to 21082
Data columns (total 10 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   количество детей в семье                    21083 non-null  float64
 1   общий трудовой стаж в днях                  21083 non-null  float64
 2   возраст клиента в годах                     21083 non-null  float64
 3   идентификатор уровня образования            21083 non-null  float64
 4   идентификатор семейного положения           21083 non-null  float64
 5   пол                                         21083 non-null  float64
 6   тип занятости                               21083 non-null  float64
 7   имел ли задолженность по возврату кредитов  21083 non-null  float64
 8   ежемесячный доход                           21083 non-null  float64
 9   категория кредита                           21083 non-null  float64
dtypes: float64

In [68]:
df2.duplicated().sum()

0

In [69]:
df2['пол'].replace({0:'Ж', 1:'М'}, inplace=True)

In [70]:
df2['тип занятости'].replace({0: 'сотрудник', 1:'компаньон', 2: 'госслужащий', 3: 'пенсионер'}, inplace=True)

In [71]:
df2['категория кредита'].replace({0:'жилищное кредитование', 1:'потребительский кредит', 2:'автокредитование', 3:'образовательный кредит', 4:'развитие бизнеса'}, inplace=True)

In [72]:
df2

,количество детей в семье,общий трудовой стаж в днях,возраст клиента в годах,идентификатор уровня образования,идентификатор семейного положения,пол,тип занятости,имел ли задолженность по возврату кредитов,ежемесячный доход,категория кредита
0,0.0,346542.0,51.8,1.0,0.0,Ж,пенсионер,0.0,71291.522491,автокредитование
1,0.0,2664.0,52.6,1.0,3.0,Ж,сотрудник,0.0,70176.435951,жилищное кредитование
2,3.0,1873.0,32.0,1.0,0.0,Ж,сотрудник,0.0,102166.458894,потребительский кредит
3,0.0,397857.0,60.8,1.0,0.0,Ж,пенсионер,0.0,97620.687042,потребительский кредит
4,0.0,1158.0,44.8,0.0,3.0,Ж,компаньон,0.0,303994.134987,автокредитование
...,...,...,...,...,...,...,...,...,...,...
21078,1.0,4529.0,43.0,1.0,1.0,Ж,компаньон,0.0,224791.862382,жилищное кредитование
21079,0.0,343937.0,67.0,1.0,0.0,Ж,пенсионер,0.0,155999.806512,автокредитование
21080,1.0,2113.0,38.0,1.0,1.0,М,сотрудник,1.0,89672.561153,жилищное кредитование
21081,3.0,3112.0,38.0,1.0,0.0,М,сотрудник,1.0,244093.050500,автокредитование


In [73]:
df2['идентификатор уровня образования'].replace({1:'среднее', 0:'высшее',2:'неоконченное высшее',3:'начальное',4:'ученая степень'},inplace=True)

In [74]:
df2['идентификатор семейного положения'].replace({0:'женат / замужем', 1:'гражданский брак', 4:'Не женат / не замужем', 3:'в разводе', 2:'вдовец / вдова'}, inplace=True)

In [75]:
out_df_uniq_propusk(df2)

*****************************************************************************************************************************


Столбец :  количество детей в семье
уникальные значения :  [0. 3. 2. 1. 4. 5.]


пропуски :  0


Описательная статистика : 
count    21083.000000
mean         0.476735
std          0.754059
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          5.000000
Name: количество детей в семье, dtype: float64


Количество значений : 
0.0    13890
1.0     4773
2.0     2041
3.0      329
4.0       41
5.0        9
Name: количество детей в семье, dtype: int64


*****************************************************************************************************************************


Столбец :  общий трудовой стаж в днях
уникальные значения :  [346542.    2664.    1873.  ...   1397.2 373996.  343937. ]


пропуски :  0


Описательная статистика : 
count     21083.000000
mean      66790.865712
std      138646.172719
min         

In [76]:
profile_posle_2 = ProfileReport(df2, title="Pandas Profiling Report")
profile_posle_2.to_file('инфо_3.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [77]:
df2['категория дохода'] = pd.qcut(df2['ежемесячный доход'], q = [0, 0.25, 0.5, 0.75, 1], labels=['маленький','умеренный','средний', 'большой'])

Исследовательский анализ данных и проверка гипотез


Для проверки последней гипотезы, посмотрим наш файлик инфо3.html, проведем категоризацию наших ежемесячных доходов, по квантилям.


In [78]:
df2['категория дохода'] = pd.qcut(df2['ежемесячный доход'], q = [0, 0.25, 0.5, 0.75, 1], labels=['маленький','умеренный','средний', 'большой'])

In [79]:
pd.qcut(df2['ежемесячный доход'], q = [0, 0.25, 0.5, 0.75, 1])

0          (20667.263, 105685.075]
1          (20667.263, 105685.075]
2          (20667.263, 105685.075]
3          (20667.263, 105685.075]
4        (201931.576, 2265604.029]
                   ...            
21078    (201931.576, 2265604.029]
21079     (146514.255, 201931.576]
21080      (20667.263, 105685.075]
21081    (201931.576, 2265604.029]
21082      (20667.263, 105685.075]
Name: ежемесячный доход, Length: 21083, dtype: category
Categories (4, interval[float64, right]): [(20667.263, 105685.075] < (105685.075, 146514.255] < (146514.255, 201931.576] < (201931.576, 2265604.029]]

In [80]:
col = ['количество детей в семье', 'идентификатор семейного положения', 'категория кредита', 'категория дохода']

In [81]:
for i in col:
    print('\n')
    print('признак : ',i)
    r = df2[[i,'имел ли задолженность по возврату кредитов']].groupby(i)
    try :
        for j in r.groups:
            print('группа : ',j)
            dolg_no = r.get_group(j)['имел ли задолженность по возврату кредитов'].value_counts()[0]
            dolg_yes = r.get_group(j)['имел ли задолженность по возврату кредитов'].value_counts()[1]
            print(f'задолженности не было у {dolg_no} человек')
            print(f'задолженность была у {dolg_yes} человек')
            print(round(dolg_yes/dolg_no*100),' % от тех, кто не имел задолженность')
            print('\n')
    except: 
        print(f'задолженности не было у {dolg_no} человек')
        print('отсутствует долг')



признак :  количество детей в семье
группа :  0.0
задолженности не было у 12828 человек
задолженность была у 1062 человек
8  % от тех, кто не имел задолженность


группа :  1.0
задолженности не было у 4330 человек
задолженность была у 443 человек
10  % от тех, кто не имел задолженность


группа :  2.0
задолженности не было у 1848 человек
задолженность была у 193 человек
10  % от тех, кто не имел задолженность


группа :  3.0
задолженности не было у 302 человек
задолженность была у 27 человек
9  % от тех, кто не имел задолженность


группа :  4.0
задолженности не было у 37 человек
задолженность была у 4 человек
11  % от тех, кто не имел задолженность


группа :  5.0
задолженности не было у 9 человек
отсутствует долг


признак :  идентификатор семейного положения
группа :  Не женат / не замужем
задолженности не было у 2505 человек
задолженность была у 273 человек
11  % от тех, кто не имел задолженность


группа :  в разводе
задолженности не было у 1103 человек
задолженность была у 84 ч

Выводы по вышеизложенной информации:
  - Зависимость наличия детей и возвратом кредита, не значительна. Самый меньший процент, от числа, у кого нет задолженности -8%.
  - Зависисмость от семейного положения присутствует, но слабая. Самый меньший процент у категории "вдова/вдовец" 7%, что не скажешь, о людях категории - не женат/не замужем.
  - Зависимость  между уровнем дохода и вовзратом кредита, также есть, но не такая явная. 
  Низкий процент у людей, у кого большой доход, у людей категории маленький доход - 9%, не так далеко отстают.
  - Автокредитование, и Кредит образования больше влияют на вовзрат в срок, остальные держутся на 8%. Жилищное кредитование, и потребительский кредит, даже при таком количестве людей, держится на 8%.